<a href="https://colab.research.google.com/github/serkansokmen/ml-workspace/blob/master/ML_Course_Week_9_RL_Keras.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip install gym pyvirtualdisplay > /dev/null 2>&1
!apt-get install -y xvfb python-opengl ffmpeg > /dev/null 2>&1

In [2]:
!apt-get update > /dev/null 2>&1
!apt-get install cmake > /dev/null 2>&1
!pip install --upgrade setuptools 2>&1
!pip install ez_setup > /dev/null 2>&1
!pip install gym[atari] > /dev/null 2>&1
!pip install pyglet==1.3.2 > /dev/null 2>&1

Requirement already up-to-date: setuptools in /usr/local/lib/python3.6/dist-packages (41.4.0)


In [3]:
import gym
from gym import logger as gymlogger
from gym.wrappers import Monitor
gymlogger.set_level(40) #error only
import tensorflow as tf
import numpy as np
import random
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
import math
import glob
import io
import base64
from collections import deque
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam

from IPython.display import HTML
from IPython import display as ipythondisplay

In [4]:
from pyvirtualdisplay import Display
display = Display(visible=0, size=(1400, 900))
display.start()

xdpyinfo was not found, X start can not be checked! Please install xdpyinfo!


<Display cmd_param=['Xvfb', '-br', '-nolisten', 'tcp', '-screen', '0', '1400x900x24', ':1005'] cmd=['Xvfb', '-br', '-nolisten', 'tcp', '-screen', '0', '1400x900x24', ':1005'] oserror=None return_code=None stdout="None" stderr="None" timeout_happened=False>

In [0]:
"""
Utility functions to enable video recording of gym environment and displaying it
To enable video, just do "env = wrap_env(env)""
"""

def show_video():
  mp4list = glob.glob('video/*.mp4')
  if len(mp4list) > 0:
    mp4 = mp4list[0]
    video = io.open(mp4, 'r+b').read()
    encoded = base64.b64encode(video)
    ipythondisplay.display(HTML(data='''<video alt="test" autoplay 
                loop controls style="height: 400px;">
                <source src="data:video/mp4;base64,{0}" type="video/mp4" />
             </video>'''.format(encoded.decode('ascii'))))
  else: 
    print("Could not find video")
    

def wrap_env(env):
  env = Monitor(env, './video', force=True)
  return env

In [0]:
# Deep Q-learning Agent
class DQNAgent:
  def __init__(self, state_size, action_size):
    self.state_size = state_size
    self.action_size = action_size
    self.memory = deque(maxlen=2000)
    self.gamma = 0.95    # discount rate
    self.epsilon = 1.0  # exploration rate
    self.epsilon_min = 0.01
    self.epsilon_decay = 0.995
    self.learning_rate = 0.001
    self.model = self._build_model()

  def _build_model(self):
    # Neural Net for Deep-Q learning Model
    model = Sequential()
    model.add(Dense(24, input_dim=self.state_size, activation='relu'))
    model.add(Dense(24, activation='relu'))
    model.add(Dense(self.action_size, activation='linear'))
    model.compile(loss='mse',
                  optimizer=Adam(lr=self.learning_rate))
    return model

  def remember(self, state, action, reward, next_state, done):
    self.memory.append((state, action, reward, next_state, done))

  def act(self, state):
    if np.random.rand() <= self.epsilon:
      return random.randrange(self.action_size)
    act_values = self.model.predict(state)
    return np.argmax(act_values[0])  # returns action

  def replay(self, batch_size):
    minibatch = random.sample(self.memory, batch_size)
    for state, action, reward, next_state, done in minibatch:
      target = reward
      if not done:
        target = (reward + self.gamma *
                  np.amax(self.model.predict(next_state)[0]))
      target_f = self.model.predict(state)
      target_f[0][action] = target
      self.model.fit(state, target_f, epochs=1, verbose=0)
    if self.epsilon > self.epsilon_min:
      self.epsilon *= self.epsilon_decay

  def load(self, name):
    self.model.load_weights(name)

  def save(self, name):
    self.model.save_weights(name)

In [7]:
env = gym.make("CartPole-v1")
env = wrap_env(env)

state_size = env.observation_space.shape[0]
action_size = env.action_space.n
agent = DQNAgent(state_size, action_size)


Instructions for updating:
If using Keras pass *_constraint arguments to layers.


In [0]:
EPISODES = 1000

done = False
batch_size = 32

# env.close()

In [10]:
%mkdir -p "save"

for e in range(EPISODES):
  state = env.reset()
  state = np.reshape(state, [1, state_size])
  for time in range(500):
    env.render()
    action = agent.act(state)
    next_state, reward, done, _ = env.step(action)
    reward = reward if not done else -10
    next_state = np.reshape(next_state, [1, state_size])
    agent.remember(state, action, reward, next_state, done)
    state = next_state
    if done:
      print("episode: {}/{}, score: {}, e: {:.2}"
            .format(e, EPISODES, time, agent.epsilon))
      break
    if len(agent.memory) > batch_size:
      agent.replay(batch_size)
  
  if e % 10 == 0:
    agent.save("./save/cartpole-dqn.h5")

env.close()
show_video()
  

episode: 0/1000, score: 46, e: 0.85
episode: 1/1000, score: 9, e: 0.81
episode: 2/1000, score: 16, e: 0.75
episode: 3/1000, score: 16, e: 0.69
episode: 4/1000, score: 8, e: 0.67
episode: 5/1000, score: 14, e: 0.62
episode: 6/1000, score: 10, e: 0.59
episode: 7/1000, score: 10, e: 0.56
episode: 8/1000, score: 13, e: 0.53
episode: 9/1000, score: 12, e: 0.5
episode: 10/1000, score: 11, e: 0.47
episode: 11/1000, score: 19, e: 0.43
episode: 12/1000, score: 10, e: 0.41
episode: 13/1000, score: 8, e: 0.39
episode: 14/1000, score: 7, e: 0.38
episode: 15/1000, score: 11, e: 0.36
episode: 16/1000, score: 9, e: 0.34
episode: 17/1000, score: 11, e: 0.32
episode: 18/1000, score: 11, e: 0.3
episode: 19/1000, score: 42, e: 0.25
episode: 20/1000, score: 85, e: 0.16
episode: 21/1000, score: 58, e: 0.12
episode: 22/1000, score: 47, e: 0.095
episode: 23/1000, score: 93, e: 0.06
episode: 24/1000, score: 52, e: 0.046
episode: 25/1000, score: 51, e: 0.036
episode: 26/1000, score: 68, e: 0.025
episode: 27/10